# 基于Transformer的古文字翻译系统

**一、项目背景介绍**

随着人类精神文明的不断提升，人们越来越重视古代文化生活，但由于古汉语字形繁复，对于非专业但对古文化感兴趣者来说造成了很大的困扰。因此想做出该项目为广大古文字爱好者提供便利。

**二、数据介绍**

该系统使用BERT研究院数据集中的中文机器翻译的数据作为训练语料，非常适合来训练Transformer机器翻译。
简体中文需要Jieba+BPE，古文字需要BPE

**三、模型介绍**

Source：![](https://ai-studio-static-online.cdn.bcebos.com/3e9c7d24c92e4740ad2f33992a9d66434d12b46ec27d43e993d205b8f29d54e1)
Target：![](https://ai-studio-static-online.cdn.bcebos.com/4b51c0336f6041ebb917ee219fe13df0d2e67ea8125b4556b910521ebd02f0c3)


   古文字翻译就是把一种古文字翻译成现代简体中文，在这里，我用的例子都是从象形文字翻译成简体中文。上面的文字用Source标记，即源语言，下面用Target标记，即目标语言，古文字翻译系统任务就是把源语言的文字翻译成目标语言的文字。在翻译一个文字的时候，会面临很多选字的问题，因为古文字中一字多形的现象比较普遍。因此采用神经网络机器翻译原理。
   我们首先通过神经网络训练出一个字向量表。源语言中的字，可以用一个one hot的向量表示。所谓one hot就是，比如上例中源语言有8个字。哪个字出现了，就把这个字标为1，其他的字标为0。比如第4个字“又”这个词是1，那么其他的都是0。这两个矩阵这么一乘，相当于一个查表的操作。就把其中这个字向量表的一列取出来了，那么这一列的向量就代表了这个字。神经网络里面所有的字都会用向量来表示。得到字的向量表示后，再经过一个循环神经网络的变换，得到另外一个向量，称为Hidden State（隐状态）。为了充分利用上下文信息，在此还要做一个双向的编码，这样既有前面的信息，也有后面的信息。所以它有了一个上下文的信息，可以进一步提高译文质量。
   编码完成以后，需要把这个源语言的字压缩到一个向量里去。这里还要用到一个注意力机制，叫Attention。这里用不同深度颜色的线去表示Attention的能量强弱，用以衡量产生目标词时，它所对应的源语言字的贡献大小。所以呢h前面又加一个α，α就表示它的一个权重。有了字的向量表示后，就掌握了整个源语言字的所有的信息。解码器就开始从左到右一个字一个字的产生目标字。第一个字产生以后，再产生第二个字，直到产生结束符EOS(End of Sentence) ，翻译就完毕了。  

**四、模型训练**

运行do_train函数， 在do_train函数中，配置优化器、损失函数，以及评价指标Perplexity；

Perplexity，即困惑度，常用来衡量语言模型优劣，即句子的通顺度，一般用于机器翻译和文本生成等领域。Perplexity越小，句子越通顺，该语言模型越好。
![](https://ai-studio-static-online.cdn.bcebos.com/d4ebdc2b0c6a4c348e2c0270e7de2fdac3fcaf6cd03040548eaf8cdd205f2ca9)


In [1]:
def do_train(args):
    if args.use_gpu:
        place = "gpu"
    else:
        place = "cpu"
    paddle.set_device(place)
    # Set seed for CE
    random_seed = eval(str(args.random_seed))
    if random_seed is not None:
        paddle.seed(random_seed)

    # Define data loader
    (train_loader), (eval_loader) = create_data_loader(args)

    # Define model
    transformer = TransformerModel(
        src_vocab_size=args.src_vocab_size,
        trg_vocab_size=args.trg_vocab_size,
        max_length=args.max_length + 1,
        n_layer=args.n_layer,
        n_head=args.n_head,
        d_model=args.d_model,
        d_inner_hid=args.d_inner_hid,
        dropout=args.dropout,
        weight_sharing=args.weight_sharing,
        bos_id=args.bos_idx,
        eos_id=args.eos_idx)

    # Define loss
    criterion = CrossEntropyCriterion(args.label_smooth_eps, args.bos_idx)

    scheduler = paddle.optimizer.lr.NoamDecay(
        args.d_model, args.warmup_steps, args.learning_rate, last_epoch=0)

    # Define optimizer
    optimizer = paddle.optimizer.Adam(
        learning_rate=scheduler,
        beta1=args.beta1,
        beta2=args.beta2,
        epsilon=float(args.eps),
        parameters=transformer.parameters())

    step_idx = 0

    # Train loop
    for pass_id in range(args.epoch):
        batch_id = 0
        for input_data in train_loader:

            (src_word, trg_word, lbl_word) = input_data

            logits = transformer(src_word=src_word, trg_word=trg_word)

            sum_cost, avg_cost, token_num = criterion(logits, lbl_word)
            
            # 计算梯度
            avg_cost.backward() 
            # 更新参数
            optimizer.step() 
            # 梯度清零
            optimizer.clear_grad() 

            if (step_idx + 1) % args.print_step == 0 or step_idx == 0:
                total_avg_cost = avg_cost.numpy()
                logger.info(
                    "step_idx: %d, epoch: %d, batch: %d, avg loss: %f, "
                    " ppl: %f " %
                    (step_idx, pass_id, batch_id, total_avg_cost,
                        np.exp([min(total_avg_cost, 100)])))

            if (step_idx + 1) % args.save_step == 0:
                # Validation
                transformer.eval()
                total_sum_cost = 0
                total_token_num = 0
                with paddle.no_grad():
                    for input_data in eval_loader:
                        (src_word, trg_word, lbl_word) = input_data
                        logits = transformer(
                            src_word=src_word, trg_word=trg_word)
                        sum_cost, avg_cost, token_num = criterion(logits,
                                                                  lbl_word)
                        total_sum_cost += sum_cost.numpy()
                        total_token_num += token_num.numpy()
                        total_avg_cost = total_sum_cost / total_token_num
                    logger.info("validation, step_idx: %d, avg loss: %f, "
                                " ppl: %f" %
                                (step_idx, total_avg_cost,
                                 np.exp([min(total_avg_cost, 100)])))
                transformer.train()

                if args.save_model:
                    model_dir = os.path.join(args.save_model,
                                             "step_" + str(step_idx))
                    if not os.path.exists(model_dir):
                        os.makedirs(model_dir)
                    paddle.save(transformer.state_dict(),
                                os.path.join(model_dir, "transformer.pdparams"))
                    paddle.save(optimizer.state_dict(),
                                os.path.join(model_dir, "transformer.pdopt"))
            batch_id += 1
            step_idx += 1
            scheduler.step()


    if args.save_model:
        model_dir = os.path.join(args.save_model, "step_final")
        if not os.path.exists(model_dir):
            os.makedirs(model_dir)
        paddle.save(transformer.state_dict(),
                    os.path.join(model_dir, "transformer.pdparams"))
        paddle.save(optimizer.state_dict(),
                    os.path.join(model_dir, "transformer.pdopt"))

In [2]:
# 读入参数
yaml_file = 'transformer.base.yaml'
with open(yaml_file, 'rt') as f:
    args = AttrDict(yaml.safe_load(f))
    pprint(args)

In [ ]:
do_train(args)

**五、模型评估**

模型最终训练的效果一般可通过测试集来进行测试，机器翻译领域一般计算BLEU值。

![](https://ai-studio-static-online.cdn.bcebos.com/ece2cc569ac2429f94edf513c831d47b388be64a6daf4ef7a4bec04b0226ce99)


In [ ]:
def do_predict(args):
    if args.use_gpu:
        place = "gpu"
    else:
        place = "cpu"
    paddle.set_device(place)

    # Define data loader
    test_loader, to_tokens = create_infer_loader(args)

    # Define model
    transformer = InferTransformerModel(
        src_vocab_size=args.src_vocab_size,
        trg_vocab_size=args.trg_vocab_size,
        max_length=args.max_length + 1,
        n_layer=args.n_layer,
        n_head=args.n_head,
        d_model=args.d_model,
        d_inner_hid=args.d_inner_hid,
        dropout=args.dropout,
        weight_sharing=args.weight_sharing,
        bos_id=args.bos_idx,
        eos_id=args.eos_idx,
        beam_size=args.beam_size,
        max_out_len=args.max_out_len)

    # Load the trained model
    assert args.init_from_params, (
        "Please set init_from_params to load the infer model.")

    model_dict = paddle.load(
        os.path.join(args.init_from_params, "transformer.pdparams"))

    # To avoid a longer length than training, reset the size of position
    # encoding to max_length
    model_dict["encoder.pos_encoder.weight"] = position_encoding_init(
        args.max_length + 1, args.d_model)
    model_dict["decoder.pos_encoder.weight"] = position_encoding_init(
        args.max_length + 1, args.d_model)
    transformer.load_dict(model_dict)

    # Set evaluate mode
    transformer.eval()

    f = open(args.output_file, "w")
    with paddle.no_grad():
        for (src_word, ) in test_loader:
            finished_seq = transformer(src_word=src_word)
            finished_seq = finished_seq.numpy().transpose([0, 2, 1])
            for ins in finished_seq:
                for beam_idx, beam in enumerate(ins):
                    if beam_idx >= args.n_best:
                        break
                    id_list = post_process_seq(beam, args.bos_idx, args.eos_idx)
                    word_list = to_tokens(id_list)
                    sequence = " ".join(word_list) + "\n"
                    f.write(sequence)
    f.close()

In [ ]:
do_predict(args)

预测结果中每行输出是对应行输入的得分最高的翻译，对于使用 BPE 的数据，预测出的翻译结果也将是 BPE 表示的数据，要还原成原始的数据（这里指 tokenize 后的数据）才能进行正确的评估

In [ ]:
# 还原 predict.txt 中的预测结果为 tokenize 后的数据
! sed -r 's/(@@ )|(@@ ?$)//g' train_dev_test/predict.txt > train_dev_test/predict.tok.txt
# BLEU评估工具来源于 https://github.com/moses-smt/mosesdecoder.git
! tar -zxf mosesdecoder.tar.gz
# 计算multi-bleu
! perl mosesdecoder/scripts/generic/multi-bleu.perl train_dev_test/ccmt2019-news.zh2en.ref*.txt < train_dev_test/predict.tok.txt

**六、总结与升华**

本项目由于作者能力有限，存在很多不足。经作者查询到的信息来看，此方面的研究信息较少，值得向该方向发展考虑。

**七、参考文献**

         [『NLP打卡营』实践课7：中英文本翻译系统_副本](https://aistudio.baidu.com/aistudio/projectdetail/3593867?forkThirdPart=1)。
         [机器翻译的流程（原理）是怎么样的？](https://www.zhihu.com/question/24588198)。
         其中部分图片来源于网络。

七、个人总结

个人介绍：重庆交通大学 航空学院 飞行器制造工程专业 2021级 本科生 江贵帮
机器学习小白

最后，诚挚的感谢飞桨创造营老师、助教和班长的帮助，同时也感谢以上资料的提供者。